In [1]:
import nidaqmx
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os
import qdac
from nidaqmx.constants import AcquisitionType as AcqType
from nidaqmx.constants import TerminalConfiguration as TermConfig
from datetime import datetime
import ipywidgets as wg
from ipywidgets import Layout
from IPython.display import display, Latex
%matplotlib widget

# Data input section
display(Latex('Input for the QDAC:'))
QDAC_port = wg.Dropdown(options=['COM1','COM2','COM3','COM4','COM5','COM6'],
    value='COM5',
    description='Port',
    layout=Layout(height='auto', width='auto'))
display(QDAC_port)

display(Latex('Inputs for the current preamp: '))

Gain = wg.Dropdown(options=['1e3','1e4','1e5','1e6','1e7','1e8','1e9','1e10','1e11'],
    value='1e9',
    description='Gain',
    layout=Layout(height='auto', width='auto'))
display(Gain)

display(Latex('Inputs for the data acquisition card: '))
Device_ID = wg.Dropdown(options=['Dev1','Dev2','Dev3'],
    value='Dev2',
    description='Device ID',
    layout=Layout(height='auto', width='auto'))
Analog_io_port = wg.Dropdown(options=['ai0','ai1','ai2','ai3'],
    value='ai1',
    description='Analog port',
    layout=Layout(height='auto', width='auto'))
Actual_Sampling_Rate = wg.Dropdown(options=['1e3','1e4','1e5','1e6'],
    value='1e6',
    description='Actual Rate',
    layout=Layout(height='auto', width='auto'))
Minimum_voltage = wg.FloatSlider(description='Min. voltage: ', min = -10, max = 10, step = 0.1, value=-10, layout=Layout(height='auto', width='auto'))
Maximum_voltage = wg.FloatSlider(description='Max. voltage: ', min = -10, max = 10, step = 0.1, value=10, layout=Layout(height='auto', width='auto'))
Acquisition_time = wg.FloatSlider(description='Acq. time: ', min = 0, max = 10, step = 0.1, value=0.5, layout=Layout(height='auto', width='auto')) #in sec

display(Device_ID, Analog_io_port, Actual_Sampling_Rate,Minimum_voltage, Maximum_voltage, Acquisition_time)

display(Latex('Inputs for data averaging: '))
Desired_Sampling_Rate = wg.Dropdown(options=['1e2','1e3','1e4','1e5','1e6'],
    value='1e3',
    description='Desired Rate',
    layout=Layout(height='auto', width='auto'))
display(Desired_Sampling_Rate)

display(Latex('Inputs for plotting data: '))
Title_of_plot = wg.Text(description='Title', layout=Layout(height='auto', width='auto'))
display(Title_of_plot)

display(Latex('Inputs for saving data: '))
Saving_data = wg.Dropdown(options=['Yes','No'],
    value='No',
    description='Save data',
    layout=Layout(height='auto', width='auto'))
Folder_name = wg.Text(description='Folder (YYYYMMDD)', style = {'description_width': 'initial'}, layout=Layout(height='auto', width='auto'))
Data_information = wg.Text(description='More info', layout=Layout(height='auto', width='auto'))
display(Saving_data,Folder_name, Data_information)


def start_data_collection():

    #Constant values
    global gain, rs_acqu, t_acqu, ns_acqu, t, current_i, rs_desired, t_desired, i_desired, ns_desired
    
    gain = float(Gain.value) # gain of the preamps
    print('gain is =',"{:.2e}".format(gain))


    # Data collection with NIDAQ at maximum sampling rate

    rs_acqu = float(Actual_Sampling_Rate.value)# sampling rate in Hz
    t_acqu = float(Acquisition_time.value) # acquisition time in second
    ns_acqu = int(rs_acqu*t_acqu)
    t = np.arange(start=0, stop=t_acqu, step=1/rs_acqu)


    with nidaqmx.Task() as acqu_task:
        # Set acquisition channels and specs
        # Change the voltage range if necessary
        acqu_task.ai_channels.add_ai_voltage_chan(Device_ID.value + '/'+ Analog_io_port.value,terminal_config=TermConfig.DIFFERENTIAL,min_val=Minimum_voltage.value,max_val=Maximum_voltage.value)
        acqu_task.timing.cfg_samp_clk_timing(rs_acqu,sample_mode=AcqType.FINITE,samps_per_chan=ns_acqu)

        # Start acquisition
        acqu_task.start()

        # Collect the data measured
        current_i = acqu_task.read(number_of_samples_per_channel=ns_acqu)
        current_i = 1e12*np.array(current_i)/gain # voltage to actual current (in pA) conversion

        # Stop acquisition
        acqu_task.stop()

        
    # Averaging to get a desired sampling rate (Similar to sampling rate conversion)
    rs_desired = float(Desired_Sampling_Rate.value) #desired sampling rate
    n_pointstoaverage = int(rs_acqu/rs_desired) # number of points to be averaged
    ns_desired = int(rs_desired*t_acqu) # total number of points for the collected signal in the desired rate
    t_desired = np.arange(start=0, stop=t_acqu, step=1/rs_desired)
    i_desired = np.zeros(ns_desired)
    
    for index in np.arange(ns_desired):
        i_desired[index] = current_i[index*n_pointstoaverage:(index+1)*n_pointstoaverage].mean()
  
    #Saving Data
    if Saving_data.value=='Yes' :
        x=t_desired
        y=i_desired
        data = np.column_stack((x, y))
        
        path=os.getcwd()
        print(os.getcwd())
        os.chdir('/Users/Measurement2/OneDrive/GroupShared/Data/Qsim/' + Folder_name.value)
        date=datetime.now()
        
        header=Data_information.value
        header+="\n"+str(date)+"\n"
        header+= "x=time(sec), y=PreampCurrent(pA)\n"
        print (header)
        
        with open('filenum.txt', 'r') as f:
          filenum = f.readline()
        
        filename='dat'+str(filenum)+'.dat'
        print(filename)

        with open(os.path.join(path,filename), 'ab') as file:
            np.savetxt(file,data,header=header,delimiter=',')    

        filenum=int(filenum)+1

        with open('filenum.txt', 'w') as f:
           f.write('%d' % filenum)

        
    plt.figure(figsize=(8,5))
    plt.plot(t, current_i)
    plt.title('Actual signal, without averaging')
    plt.ylabel('Preamp current (pA)')
    plt.xlabel('time (sec)')
    
    plt.figure(figsize=(8,5))
    plt.plot(t_desired, i_desired)
    plt.title("Averaged signal: " + Title_of_plot.value)
    plt.ylabel('Preamp current (pA)')
    plt.xlabel('time (sec)')
    
    

def plot_actual_spectrum():
    plt.figure(figsize=(8,5))

    f,II_den=signal.periodogram(current_i,fs=rs_acqu,window='flattop',scaling='density',return_onesided=True)
    plt.loglog(f,np.sqrt(II_den))
    plt.xlabel('f [Hz]')
    plt.ylabel('$\sqrt{S_{ii}(f)}\, (pA / \sqrt{Hz})$')
    plt.title('Actual spectrum')


def plot_time_averaged_spectrum():
    plt.figure(figsize=(8,5))

    f,II_den=signal.periodogram(i_desired,fs=rs_desired,window='flattop',scaling='density',return_onesided=True)
    plt.loglog(f, np.sqrt(II_den))
    plt.xlabel('f [Hz]')
    plt.ylabel('$\sqrt{S_{ii}(f)}\, (pA / \sqrt{Hz})$')
    plt.title('Spectrum of the time averaged signal')
    
def generate_IV(voltage_channel_number, min_voltage, max_voltage, n_of_voltage_points):
    #Constant values
    global gain, rs_acqu, t_acqu, ns_acqu, t, current_i, rs_desired, t_desired, i_desired, ns_desired
    
    gain = float(Gain.value) # gain of the preamps
    print('gain is =',"{:.2e}".format(gain))


    # Data collection with NIDAQ at maximum sampling rate

    rs_acqu = float(Actual_Sampling_Rate.value)# sampling rate in Hz
    t_acqu = float(Acquisition_time.value) # acquisition time in second
    ns_acqu = int(rs_acqu*t_acqu)
    t = np.arange(start=0, stop=t_acqu, step=1/rs_acqu)
    
    
    N = n_of_voltage_points
    voltages = np.linspace(min_voltage, max_voltage, num=N)
    currents = np.zeros(N)

    for i in np.arange(N):
        with qdac.qdac(QDAC_port.value) as q:
            print('Setting Voltage at channel 1 is: ', q.setDCVoltage(voltage_channel_number, voltages[i]), ' V')
            
            with nidaqmx.Task() as acqu_task:
                acqu_task.ai_channels.add_ai_voltage_chan(Device_ID.value + '/'+ Analog_io_port.value,terminal_config=TermConfig.DIFFERENTIAL,min_val=Minimum_voltage.value,max_val=Maximum_voltage.value)
                acqu_task.timing.cfg_samp_clk_timing(rs_acqu,sample_mode=AcqType.FINITE,samps_per_chan=ns_acqu)

                # Start acquisition
                acqu_task.start()

                # Collect the data measured
                current_i = acqu_task.read(number_of_samples_per_channel=ns_acqu)
                current_i = 1e9*np.array(current_i)/gain # voltage to actual current (in nA) conversion


                # Stop acquisition
                acqu_task.stop()

                currents[i] = current_i.mean()

    with qdac.qdac('COM5') as q:
            print('All done! Setting Voltage at channel ',  voltage_channel_number, ' to zero V.' )
            q.setDCVoltage(voltage_channel_number, 0)

    plt.figure()
    plt.plot(voltages*1e3, currents, '-*')
    plt.xlabel('Voltage (mV)')
    plt.ylabel('Current (nA)')
    R = np.polyfit(voltages, currents, 1)
    plt.title('Linear fit: Resistance is '+ str(1e6/R[0]) + ' kilo-Ohm')

<IPython.core.display.Latex object>

Dropdown(description='Port', index=4, layout=Layout(height='auto', width='auto'), options=('COM1', 'COM2', 'CO…

<IPython.core.display.Latex object>

Dropdown(description='Gain', index=6, layout=Layout(height='auto', width='auto'), options=('1e3', '1e4', '1e5'…

<IPython.core.display.Latex object>

...

Text(value='', description='Title', layout=Layout(height='auto', width='auto'))

<IPython.core.display.Latex object>

Dropdown(description='Save data', index=1, layout=Layout(height='auto', width='auto'), options=('Yes', 'No'), …

Text(value='', description='Folder (YYYYMMDD)', layout=Layout(height='auto', width='auto'), style=DescriptionS…

Text(value='', description='More info', layout=Layout(height='auto', width='auto'))

In [ ]:
generate_IV(voltage_channel_number=1, min_voltage=-5e-3, max_voltage=5e-3, n_of_voltage_points=20)

In [2]:
start_data_collection()

gain is = 1.00e+07


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …